In [0]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)

In [4]:
df = pd.read_csv(path/'texts.csv')
print(df.head())
print(df.shape)

      label                                               text  is_valid
0  negative  Un-bleeping-believable! Meg Ryan doesn't even ...     False
1  positive  This is a extremely well-made film. The acting...     False
2  negative  Every once in a long while a movie will come a...     False
3  positive  Name just says it all. I watched this movie wi...     False
4  negative  This movie succeeds at being one of the most u...     False
(1000, 3)


In [14]:
# split into train and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size = 0.2)
print(train_df.shape[0])
print(test_df.shape[0])

train_df = train_df.reset_index(drop = True)
print(train_df.head())

800
200
      label                                               text  is_valid
0  negative  Following the disasterous Revolution, this fil...      True
1  positive  Excellent view of a mature woman, that is goin...     False
2  negative  TV does influence society...just look at the s...     False
3  negative  You like beautiful girls? Yeah me too. What is...     False
4  positive  well "Wayne's World" is long gone and the year...      True


In [32]:
# process a review

from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer

import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

def process_review(text):
  sents = sent_tokenize(text)
  words_pos = []
  for s in sents:
    words_pos.extend(pos_tag(word_tokenize(s)))
    filter_words_pos = [w.lower() for (w,pos) in words_pos if pos[0] in ['V', 'J', 'N', 'R']] 

    return ' '.join(filter_words_pos)
  # remove
  # tokenize, POS
  # retain/remove POS

processed_reviews = [process_review(text) for text in train_df['text']]
print(len(processed_reviews))



800


In [41]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range =(1,2), analyzer = 'word', min_df = 3)
counter_train = vect.fit_transform(processed_reviews)

print(counter_train.shape)

(800, 955)


In [43]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_idf_vect = TfidfTransformer(sublinear_tf = True)
tf_idf_train = tf_idf_vect.fit_transform(counter_train)
print(tf_idf_train.shape)

(800, 955)


In [44]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB()
naive_bayes.fit(counter_train, train_df['label'])


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [49]:
test_reviews = [process_review(text) for text in test_df['text']]
counter_test = vect.transform(test_reviews)
print(counter_test.shape)

(200, 955)


In [0]:
predict_label = naive_bayes.predict(counter_test)

In [52]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_df['label'], predict_label)
print(accuracy)

0.66


In [54]:
predict_text = ["this is a good movie", "this is a bad movie", "this is not a bad movie", "this is not a good movie", 
                                            "this is an ok movie", "I really liked this movie. bad movie", 
                                            "bad movie. But I really liked the ending"]
predict_reviews = [process_review(text) for text in predict_text]
counter_predict = vect.transform(predict_reviews)
print(counter_predict.shape)  

print(naive_bayes.predict(counter_predict))              

(7, 955)
['positive' 'negative' 'negative' 'positive' 'negative' 'positive' 'negative']
